## 问题1：[最长连续序列](https://leetcode.cn/problems/longest-consecutive-sequence/description/)
### 题目
给定一个未排序的整数数组nums, 找出数字连续的最长序列(不要求序列元素在原数组中连续)的长度。

请你设计并实现时间复杂度为 O(n) 的算法解决此问题。

### 分析
- 由于需要时间复杂度为n的算法, 所以我们不能排序. 为解决序列问题需要, 明确每一个数时候能对最终的答案有贡献, 使用哈希表来记录每一个数字所在的连续序列长度。比如，如果存在2,3,4的连续序列, 则哈希表中2,3,4对应的值即为3(表示这些值所在连续序列的长度)
- 遍历nums, 当出现i时, 判断i是否已经被使用过了(已经在哈希表中), 如果是则跳过。如果不是则需要加入到hash表中。
    - 对于新加入的值我们需要考察是否该值会使得某两个区间合并，比如已经存在[1，3]区间和[5,6]区间，如果加入4，则可以将连续区间扩大为[1，6]
    - 因此我们只需要考察哈希表中3，5的值，将hash(3) + hash(5) + 1 即为合并后的长度
    - 注意: 我们此时需要更新区间[1,6]中每一个对应的hash值吗？其实不用我们发现我们只用更新端点的值即可(只用更新1,6对应的值), 因为2，3，4，5这些值在下一次碰见时是没有任何作用的

In [ ]:
def process(nums):
    hash_dict = {}
    ans = 0
    for i in nums:
        if i not in hash_dict:
            left_inter = hash_dict.get(i-1, 0)
            right_inter = hash_dict.get(i+1, 0)
            current_inter = left_inter + right_inter + 1
            # 更新最大值
            ans = max(ans, current_inter)
            # 更新端点值
            hash_dict[i] = current_inter
            hash_dict[i - left_inter] = current_inter
            hash_dict[i + right_inter] = current_inter
            print(hash_dict)
    return ans

nums = [100,4,200,1,3,2]
process(nums)

## 问题2：[字母异位词分组](https://leetcode.cn/problems/group-anagrams/description/)
### 题目
给你一个字符串数组，请你将 字母异位词 组合在一起。可以按任意顺序返回结果列表。

字母异位词: 是由重新排列源单词的所有字母得到的一个新单词。
### 分析
- 使用一种特殊的哈希方法, 将每个元素映射成为一个则26位的字符串: ord(字母)-97表示位置, 如果出现了1次则对应位置上+1, 最后返回一个字符串，表示这个单词的特征, 最后可以使用tuple([1,2,3])的方式实现作为键.

In [ ]:
from collections import defaultdict

def hash_word(word):
    ans = [0] * 26
    for i in word:
        ans[ord(i)-97] += 1
    return str(ans)

def process(strs):
    hash_map = defaultdict(list)
    for word in strs:
        hash_map[hash_word(word)].append(word)
    
    return [value for _, value in hash_map.items()]

strs = ["bdddddddddd","bbbbbbbbbbc"]
process(strs)
hash_word("bdddddddddd")

## 问题3：[移动零](https://leetcode.cn/problems/move-zeroes/description/)
### 题目
给定一个数组 nums，编写一个函数将所有 0 移动到数组的末尾，同时保持非零元素的相对顺序。
请注意 ，必须在不复制数组的情况下原地对数组进行操作。

### 分析
- 遍历时采用remove(0)（每次只会去除第一个数字） + append(0)将0移动到末尾
- **注意**: 不可以在遍历nums对nums进行修改！！因此我们需要先遍历完成记录有多少个0
```python
a = [1,2,3,4]
iterator = iter(a)
print(next(iterator)) # 此时游标来到了index=0
a.pop(0)              # 由于删除了0位置, 因此此时的a为[2,3,4]
print(next(iterator)) # 此时游标来到了index=1, 因此打印的时[2，3，4]的index=1的位置即为3
```

### 优化
- 从左往右遍历时, 如果去修改nums即立刻pop(index) + append(0), 会导致index偏移，由于我们删除后会添加，因此我们从右往左遍历，删除和添加后index向前移动不会出现跳过某些数字的情况


In [ ]:
def process(nums):
    count = 0
    for num in nums:
        if num == 0:
            count += 1
    for _ in range(count):
        nums.remove(0)
        nums.append(0)
        
    return nums

# 优化
def process(nums):
    right = len(nums)-1
    while right >= 0:
        if nums[right] == 0:
            nums.pop(right)
            nums.append(0)
        right -= 1

## 问题3：[]()
### 题目


### 分析

## 问题3：[]()
### 题目


### 分析

## 问题3：[]()
### 题目


### 分析